This is the powershell part of the demo. In this part I am going to create the configuration for Data API Builder.

We start with how you can install it for your developement. Before we install we must have a few things:

> 1)Your computer for development
> 
> 2. A SQL Database (you can a docker image or a VM)
>     
> 3. A folder structure
>     

We assume you have the first two in place.

The folder structure I would prefer to build is created under GIT

We will create the structure if it not exists already and jump into the folder for dab

In [14]:
# Create Folder structure
$basepath = "c:\GIT2"
$Projectname = "SWA-DAB-SETUP"

if (!(test-path $basepath)){new-item -path $basepath -itemtype Directory -force | Out-Null; write-output "$Basepath Created" }
if (test-path $basepath){
    set-location $basepath
    if (!(test-path $Projectname)){new-item -path $ProjectName -itemtype Directory -force | Out-Null; write-output "$ProjectName Created" }
    if (test-path $Projectname)
        {
        
        new-item -path "$ProjectName\BUILD" -itemtype Directory -force | Out-Null; write-output "$ProjectName\BUILD Created"

        new-item -path "$ProjectName\docs" -itemtype Directory -force | Out-Null; write-output "$ProjectName\DOCS Created"
        new-item -path "$ProjectName\src" -itemtype Directory -force | Out-Null; write-output "$ProjectName\SRC Created"

        new-item -path "$ProjectName\src\API" -itemtype Directory -force | Out-Null; write-output "$ProjectName\DOCS Created"
        new-item -path "$ProjectName\src\swa-db-conncections" -itemtype Directory -force | Out-Null; write-output "$ProjectName\DOCS Created"
        new-item -path "$ProjectName\src\web" -itemtype Directory -force | Out-Null; write-output "$ProjectName\DOCS Created"

        new-item -path "$ProjectName\TEST" -itemtype Directory -force | Out-Null; write-output "$ProjectName\TEST Created"
        }
        else
        {write-output "$ProjectName already  exists"}
    }
    else
    {
    write-output "$basepath already exists"
    }

set-location "$basepath\$ProjectName\src\swa-db-conncections"


c:\GIT2 Created
SWA-DAB-SETUP Created
SWA-DAB-SETUP\BUILD Created
SWA-DAB-SETUP\DOCS Created
SWA-DAB-SETUP\SRC Created
SWA-DAB-SETUP\DOCS Created
SWA-DAB-SETUP\DOCS Created
SWA-DAB-SETUP\DOCS Created
SWA-DAB-SETUP\TEST Created


In [ ]:
Now that we have the Git structure, it is time to configure DAB. First step would be to install it.

Installation can be achieved in two ways:
- Download the nuget package
- use the version in Visual Studio (professional or Enterprise has it available. Not sure about community edition)



In [18]:
# Set TLS1.2
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
# Download the file if required from the nuget library

Invoke-WebRequest -Uri "https://www.nuget.org/api/v2/package/Microsoft.DataApiBuilder/0.9.6-rc" -OutFile "C:\GIT2\microsoft.dataapibuilder.0.9.6-rc.nupkg"
#The following will install the file
. dotnet tool install -g --add-source C:\GIT2\ microsoft.DataApiBuilder

Tool 'microsoft.dataapibuilder' is already installed.


Now that we have DAB installed and a folderstructure, it is time to setup a connection to your database. For that you need to update the servername, databasename, userid and pwd in the following codeblock

In [ ]:
# Setup DAB, you should copy this to the terminal window below
$InstanceName   = "ToBo-SQL2022"
$DatabaseName   = "BikeStores_2022"
$UserID         = "MonkeyUser"
$PWD            = "UnH@ckable!"


# Set location to the git folder for db connections
set-location "C:\GIT2\SWA-DAB-SETUP\src\swa-db-conncections"

#from this location we can run the command to create a dab-config.json file

. dab init --database-type "mssql" --connection-string "Server=$InstanceName;Database=$DatabaseName;User ID=$UserID;Password=$PWD;TrustServerCertificate=true" --host-mode "Development“


>> 

Now that we have the default configuration, it is time to add some stored procedures and views to the configuration

In [ ]:
# First we are going to create some API endpoints based on some stored procedures
# make sure we are in the right folders
set-location "C:\GIT2\SWA-DAB-SETUP\src\swa-db-conncections"
# wrong example (table as an endpoint), just to show that it works
dab add productionproducts --source production.products --permissions “anonymous:*”
dab add salesstaff --source sales.staffs --permissions "anonymous:*"
dab add salesstores --source sales.stores --permissions "anonymous:*"

# Stored procedure
dab add OrderBook --source dbo.View_Orderbook --source.type View --source.key-fields "order_id","store_id" --permissions "anonymous:read"

dab add GetSalesData --source dbo.csp_SalesData --source.type "stored-procedure" source.params "Year" --permissions "anonymous:execute" --rest.methods "get" --graphql.operation "query"


After adding some of these, it is time to start the DAB engine...

After DAB is started, we can verify that it is working by:

1. Going to [https://localhost:5001/Swagger](https:\localhost:5001\Swagger)
    
2. Going to [https://localhost:5001/GraphQL](https:\localhost:5001\GraphQL)
    

<span style="color: var(--vscode-foreground);">Example of GraphQL:</span>  

<span style="color: #048fcf;">query</span> <span style="color: #69a6e4;">{</span><span style="color: #21c996;">SalesStores</span><span style="color: #69a6e4;">:</span> <span style="color: #e04a98;">salesstores</span> <span style="color: #69a6e4;">{</span>

    <span style="color: #e04a98;">items</span>

    <span style="color: #69a6e4;">{</span>

      <span style="color: #e04a98;">store_name</span>

      <span style="color: #69a6e4;">}</span>

    <span style="color: #69a6e4;">}</span>

<span style="color: #69a6e4;">}</span>

<span style="color: #69a6e4;"><br></span>

Another way to test if our API's are working is by some sample Powershell Code

In [ ]:
# First set some basics
$baseURI = "https://localhost:5001/api"

# Retrieve an order from the past...
# state should be added.
$Utrecht =@{
  "store_name"= "PPBikes Utrecht"
  "phone"= "+31 30 151 52 53"
  "email"= "Groningen@ppbikes"
  "street"= "Croeselaan 18"
  "city"= "Utrecht"
  "country"= "Nederland"
 
  "zip_code"= "3521 CB" 
} |Convertto-json

$utrecht = (Invoke-RestMethod -Uri  "$($baseURI)/salesstores" -Method post -Body $Utrecht -ContentType "application/json").value.store_id

$Humme = @{
  "first_name"= "Marjolein"
  "last_name"= "Humme"
  "email"= "Marjolein@ppikes.com"
  "phone"= "030 151 5299"
  "active"= "1"
  "store_id"= $Utrecht
  "manager_id" = "1"
  } | convertto-json


$Humme = (Invoke-RestMethod -Uri  "$($baseURI)/salesstaff" -Method post -Body $Humme -ContentType "application/json").value.staff_id




# Get method...
(Invoke-RestMethod -Uri "$($baseURI)/GetSalesData" -method get -contenttype "application/json").value

What if we are going to change this to the cloud database?
We should then: 
1. Stop DAB (Ctrl-C)
    
2. Rename dab-config.json to dab-config-local.json
    
3. Rename dab-config-azure.json to dab-config-azure.json
    
4. start DAB
    
Or...
run dab with an option of the config file name... maybe more helpfull
<span style="color: rgb(22, 22, 22); font-family: SFMono-Regular, Consolas, &quot;Liberation Mono&quot;, Menlo, Courier, monospace; white-space: pre; background-color: rgb(242, 242, 242);">dab start -c dab-config-azure.json</span>